[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adan-rs/amd/blob/main/notebooks/13_Comparacion_de_grupos.ipynb)

# Comparación de grupos
"*La mitad de mi gasto en publicidad no sirve para nada. El problema es que no sé qué mitad" atribuida a John Wanamaker(1838-1922)*

En el análisis de datos aplicado a los negocios, uno de los objetivos más comunes es comparar grupos para tomar decisiones fundamentadas. Estas comparaciones permiten evaluar si existen diferencias significativas entre segmentos de clientes, campañas, productos o procesos, y son clave para validar hipótesis estratégicas. Existen tres tipos principales de comparaciones estadísticas entre grupos: comparación de proporciones, comparación de medias y comparación de varianzas.

## Pruebas A/B
Un caso muy común de lo anterior en negocios, especialmente en marketing digital y comercio electrónico, es el uso de pruebas A/B, que consisten en comparar dos versiones de una misma estrategia (por ejemplo, diseño de página, mensaje publicitario, precio, etc.).
Estas pruebas generalmente se centran en comparar:
- Medias: como el tiempo promedio en la página o el ticket de compra.
- Proporciones: como la tasa de conversión o de clics (CTR).
La lógica detrás de las pruebas A/B es precisamente la comparación de grupos para determinar si la diferencia observada es estadísticamente significativa y no producto del azar. Esto permite optimizar decisiones basadas en datos reales del comportamiento de los usuarios.
  
En la siguiente liga se muestran varios ejemplos de aplicaciones prácticas:
https://goodui.org/leaks/. Este tipo de experimentos ejemplifica la aplicación de técnicas que veremos en esta sesión relacionadas con la comparación de dos o más grupos

El archivo "cookie_cats.csv" contiene información de 90,189 jugadores. Cuando un jugador instala el juego empieza en el nivel 30, sin embargo, en la empresa que diseñó el juego han propuesto que el jugador empiece en el nivel 40. Para evaluar ambas opciones se diseñó una prueba A/B en la cual los jugadores fueron asignados aleatoriamente para comenzar en el nivel 30 (control) o en el nivel 40 (tratamiento).

<div style="text-align: center;">
<img src="../recursos/cookie_cats.jpg" alt="Cookie cats" width="300" height="200">
</div>

Las variables en el dataset son:

- userid: identificador del jugador.
- version: gate30, empieza en el nivel 30; gate40: empieza en el nivel 40.
- sum_gamerounds: número de rondas jugadas durante los primeros 14 días.
- retention_1: el jugador jugó un día después de instalar el juego.
- retention_7_ el jugador jugó después de 7 días de haber instalado el juego.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://github.com/adan-rs/amd/raw/main/data/cookie_cats.csv')
df.info()

In [ ]:
df.describe().T

In [ ]:
# Eliminación de datos atípicos
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
df = remove_outliers_iso_forest(df, columns=['sum_gamerounds'])
df.info()

## Prueba z para dos proporciones
*¿Para qué se utiliza?*
La prueba z para dos proporciones se emplea para determinar si existe una diferencia estadísticamente significativa entre las proporciones de éxito de dos grupos independientes. 
Ejemplos en el área de negocios son:
- ¿La proporción de clientes que hacen clic en un anuncio difiere entre dos versiones del mismo?
- ¿La tasa de conversión varía entre usuarios nuevos y recurrentes?
- ¿Hay diferencia en la proporción de empleados que renuncian entre dos sucursales?

*Variables consideradas:*
Se analiza una variable categórica dicotómica (como “sí/no”, “éxito/fracaso”, “aprueba/no aprueba”) en dos grupos independientes. La variable puede estar en escala nominal o binaria codificada como 0 y 1.

*¿Cómo funciona?*
Para cada grupo se calcula la proporción muestral de éxitos en el primer grupo y el segundo grupo.

Se plantea la hipótesis nula de que las proporciones poblacionales son iguales. Bajo esta hipótesis, se calcula una proporción combinada (pooled proportion):
Luego se utiliza esta proporción conjunta para calcular el estadístico z, que sigue una distribución normal estándar bajo la hipótesis nula.

Hipótesis planteadas (prueba bilateral):
- H<sub>0</sub>: p=p<sub>0</sub> (Hipótesis nula)
- H<sub>1</sub>: p ≠ p<sub>0</sub> (Hipótesis alternativa)

También es posible plantear una prueba unilateral si se desea evaluar si una proporción es mayor o menor que la otra.

*Supuestos o requisitos principales:*
- Las muestras deben ser aleatorias e independientes entre sí.
- La variable debe ser dicotómica (solo dos posibles resultados).
- Cada grupo debe tener un número suficiente de observaciones para garantizar que la distribución de las proporciones se aproxime a la normal (regla común: np ≥ 5, y lo mismo para el segundo grupo).

*Criterio de decisión:*
Se calcula el estadístico z y su correspondiente valor p. Si el valor p es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula, concluyendo que existe una diferencia significativa entre ambas proporciones.

Para calcular las proporciones conviene obtener una tabla con el conteo de cada caso. Realicemos una tabla de contigencia o tabla cruzada. 

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

def compare_proportions(data, group_column, category_column,
                       success_category=None, alpha = 0.05):
    """
    Realiza e interpreta una prueba z bilateral para comparar proporciones entre grupos.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        category_column: Nombre de la columna con las categorías a comparar
        success_category: Categoría a considerar como "éxito" (si es None, 
                         usa la primera categoría encontrada)
        alpha: Nivel de significancia (predeterminado: 0.05)    
    Returns:
        p-valor
    """
    # Identificar categorías
    categorias = data[category_column].unique()  # Obtener valores únicos
    if len(categorias) != 2:                     # Sólo se comparan dos grupos
        raise ValueError(f"La columna {category_column} debe tener exactamente dos categorías")
    
    # Determinar categoría de éxito
    if success_category is None:
        success_category = categorias[0]
    elif success_category not in categorias:
        raise ValueError(f"La categoría '{success_category}' no existe en los datos")
    
    # Convertir a binario (1 para success_category, 0 para la otra)
    data_binaria = data.copy()
    data_binaria[category_column] = (data[category_column] == success_category).astype(int)
    
    # Calcular estadísticos
    conteos = data_binaria.groupby(group_column)[category_column].agg(['sum', 'count'])
    proporciones = (conteos['sum'] / conteos['count']).round(4)
    
    # Realizar prueba z
    stat, p_value = proportions_ztest(conteos['sum'], conteos['count'])
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Estadístico z: {stat:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      f"Proporciones de '{success_category}' por grupo:\n")
    for grupo in proporciones.index:
        n = conteos.loc[grupo, 'count']
        prop = proporciones[grupo]
        interpretacion += f"{grupo}: {prop:.1%} (n={n})\n"    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas)" if es_significativo else "")

    print(f'Categoria de exito: {success_category}')
    print(interpretacion)

    return p_value

In [ ]:
results = compare_proportions(df, 
                              group_column='version',
                              category_column='retention_7',
                              success_category=True)

Ejemplo de resultados:
>Se realizó una prueba z para comparar las proporciones de clics entre dos versiones de un anuncio publicitario (versión A y versión B) como parte de una prueba A/B. En la versión A, el 18.2% de los usuarios hizo clic en el anuncio (n = 275), mientras que en la versión B, la proporción de clics fue del 24.7% (n = 290). La prueba z reveló una diferencia estadísticamente significativa entre las proporciones (z = 2.15, p = 0.0316), lo que indica que la versión B generó una tasa de clics significativamente mayor. Este resultado sugiere que la versión B es más efectiva y debe considerarse para su implementación en la campaña principal.

## Prueba t para dos muestras independientes

*¿Para qué se utiliza?*
La prueba t para dos muestras independientes se utiliza para evaluar si existe una diferencia estadísticamente significativa entre las medias de dos grupos independientes. Es una de las pruebas más comunes para comparar resultados entre dos condiciones, tratamientos o poblaciones distintas.
Ejemplos de negocios son:
- ¿El ingreso promedio de los clientes es diferente entre dos regiones geográficas?
- ¿Una nueva capacitación mejora el promedio de productividad del personal?
- ¿El ticket promedio de venta varía entre hombres y mujeres?

*Variables consideradas:*
Se analiza una variable cuantitativa continua (de intervalo o razón) medida en dos grupos independientes. Ejemplos incluyen comparar el ingreso promedio entre hombres y mujeres, o el nivel de colesterol entre pacientes tratados con dos medicamentos distintos.

*¿Cómo funciona?*
La prueba compara las medias muestrales de ambos grupos y evalúa si la diferencia observada puede atribuirse al azar, bajo el supuesto de que las medias poblacionales son iguales.

Dependiendo de si se asume igualdad de varianzas o no, se utiliza la versión clásica de Student o la versión de Welch (más robusta ante varianzas desiguales).
Hipótesis planteadas (prueba bilateral):
- Hipótesis nula (H₀): Las medias poblacionales son iguales (μ1=μ2)
- Hipótesis alternativa (H₁): Las medias poblacionales son diferentes (μ1≠μ2 )
También es posible plantear pruebas unilaterales si se quiere evaluar si una media es mayor o menor que la otra.

*Supuestos o requisitos principales:*
1.	Escala adecuada: La variable debe estar medida en escala de intervalo o de razón.
2.	Distribución normal: La variable debe estar distribuida normalmente en cada grupo. Este supuesto puede relajarse si el tamaño de muestra en cada grupo es mayor a 30 (gracias al teorema del límite central).
3.	Independencia: Las observaciones dentro y entre los grupos deben ser independientes.
4.	Homogeneidad de varianzas (opcional): Aunque la prueba clásica asume varianzas iguales, si se duda de este supuesto se puede aplicar la versión de Welch, que no requiere varianzas iguales.

*Criterio de decisión:*
Se calcula el estadístico t, el cual se compara con una distribución t de Student con un número de grados de libertad determinado por el método elegido (Student o Welch). Si el valor p es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula y se concluye que hay una diferencia significativa entre las medias de los dos grupos.


Los promedios también son una métrica comúnmente utilizada en las pruebas A/B. Comparemos las rondas de juego en promedio por cada grupo en el experimento.

In [ ]:
# Estadística descriptiva
df.groupby('version').agg({'sum_gamerounds':'mean'})

Consideraciones previas para realizar una prueba t:  
- la variable debe tener escala de medición de intervalo o de razón
- la variable debe tener una distribución normal o la muestra debe ser grande (mayor a 30)
- las observaciones deben ser independientes.

Planteamiento de hipótesis:  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$

In [ ]:
from scipy.stats import ttest_ind

def compare_means(data, group_column, measure_column,
                  group1=None, group2=None,
                  equal_var: bool = True,
                  alpha = 0.05):
    """
    Realiza e interpreta una prueba t bilateral para comparar medias entre grupos.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        group1: Primer grupo a comparar (si es None, usa el primero encontrado)
        group2: Segundo grupo a comparar (si es None, usa el segundo encontrado)
        equal_var: Si es True, asume varianzas iguales (t de Student)
                   Si es False, no asume varianzas iguales (t de Welch)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos = data[group_column].unique()    # Obtener valores únicos
    if len(grupos) != 2:
        raise ValueError(f"La columna {group_column} debe tener exactamente dos grupos")  
    group1 = group1 if group1 is not None else grupos[0]
    group2 = group2 if group2 is not None else grupos[1]
    
    # Obtener datos de cada grupo
    datos_group1 = data[data[group_column] == group1][measure_column]
    datos_group2 = data[data[group_column] == group2][measure_column]
    
    # Calcular estadísticos descriptivos
    stats = pd.DataFrame({
        'n': [len(datos_group1), len(datos_group2)],
        'media': [datos_group1.mean(), datos_group2.mean()],
        'std': [datos_group1.std(), datos_group2.std()]}, 
        index=[group1, group2])
    
    # Realizar prueba t
    t_stat, p_value = ttest_ind(datos_group1, datos_group2, equal_var=equal_var)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"{'Prueba t de Student' if equal_var else 'Prueba t de Welch'}\n"
                      f"Estadístico t: {t_stat:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      "Estadísticos descriptivos:\n")
    
    for grupo in [group1, group2]:
        n = stats.loc[grupo, 'n']
        media = stats.loc[grupo, 'media']
        std = stats.loc[grupo, 'std']
        interpretacion += f"{grupo}: media={media:.2f}, std={std:.2f}, n={n}\n"
    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas entre las medias)" 
                                           if es_significativo else "")
    
    print(interpretacion)
    return p_value

In [ ]:
resultados = compare_means(df, 'version', 'sum_gamerounds')

El p-valor reportado corresponde a una prueba bilateral, es decir, cuando no se anticipa si la diferencia será positiva o negativa

Si la muestra es pequeña (p<30) se requiere evaluar la normalidad en la distribución de los datos. Existen varias pruebas para evaluar la normalidad de los datos. La prueba Shapiro-Wilk es una de las más utilizadas.  
- Hipótesis nula (H0): Los datos provienen de una distribución normal  
- Hipótesis alternativa (H1): Los datos no provienen de una distribución normal.  

Ejemplo de reporte de resultados:
>Se aplicó una prueba t para muestras independientes con el fin de comparar el promedio de productividad mensual (en unidades producidas) entre dos turnos de trabajo en una planta manufacturera: turno matutino y turno vespertino. La muestra incluyó a 35 trabajadores por turno. El promedio de productividad en el turno matutino fue de 128.4 unidades (DE = 14.6), mientras que en el turno vespertino fue de 119.7 unidades (DE = 15.2). La prueba t arrojó un resultado de t(68) = 2.56, con un valor p = 0.0129, lo que indica que la diferencia entre ambos grupos es estadísticamente significativa al nivel del 5%. En consecuencia, se concluye que el turno matutino presenta una mayor productividad promedio.


## Prueba de Levene
*¿Para qué se utiliza?*
La prueba de Levene se emplea para evaluar si dos o más grupos presentan varianzas poblacionales iguales. Es una herramienta fundamental para comprobar el supuesto de homogeneidad de varianzas, necesario en muchas pruebas estadísticas como el ANOVA o la prueba t para dos muestras independientes (versión clásica). Comparar varianzas también permite saber si la dispersión o variabilidad de un proceso es distinta entre grupos. Esto es importante para evaluar consistencia, control de calidad o riesgo.

Ejemplos en el área de negocios son:
- ¿Una nueva máquina reduce la variabilidad en los tiempos de producción?
- ¿Los tiempos de entrega son más consistentes en una zona geográfica respecto a otra?
- ¿Las variaciones en el rendimiento financiero son distintas entre dos líneas de negocio?

*Variables consideradas:*
- Una variable cuantitativa continua (de intervalo o razón), que se desea comparar entre grupos.
- Una variable categórica que define los grupos (nominal u ordinal).

*¿Cómo funciona?*
La prueba transforma los datos en desviaciones absolutas respecto a la media (o mediana) del grupo, y luego compara esas desviaciones mediante un análisis de varianza (ANOVA). El estadístico de Levene evalúa si estas desviaciones son significativamente distintas entre los grupos.

*Hipótesis planteadas:*
- Hipótesis nula (H₀): Las varianzas de los grupos son iguales (existe homogeneidad de varianzas).
- Hipótesis alternativa (H₁): Al menos una de las varianzas difiere (existe heterogeneidad de varianzas).

*Supuestos o requisitos principales:*
- La variable cuantitativa debe estar medida en escala continua (intervalo o razón).
- Las observaciones deben ser independientes dentro y entre grupos.
- A diferencia de otras pruebas (como F de Fisher), no requiere que los datos provengan de distribuciones normales, lo que hace a Levene más robusta ante desviaciones de la normalidad.

*Criterio de decisión*: Se revisa el valor p asociado al estadístico de Levene.
- Si p > 0.05, no se rechaza la hipótesis nula: se asume que las varianzas son iguales.
- Si p ≤ 0.05, se rechaza la hipótesis nula: hay evidencia de que al menos un grupo tiene una varianza distinta.


In [ ]:
from scipy.stats import levene

def compare_variances(data, group_column, measure_column, groups=None, alpha=0.05):
    """
    Realiza e interpreta una prueba de Levene para comparar varianzas entre múltiples grupos.   
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        groups: Lista de grupos a comparar (si es None, usa todos los grupos encontrados)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos_disponibles = data[group_column].unique()
    if len(grupos_disponibles) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    
    # Usar grupos especificados o todos los disponibles
    grupos = groups if groups is not None else grupos_disponibles
    
    # Verificar que todos los grupos especificados existen
    grupos_invalidos = set(grupos) - set(grupos_disponibles)
    if grupos_invalidos:
        raise ValueError(f"Grupos no encontrados en los datos: {grupos_invalidos}")
    
    # Obtener datos de cada grupo
    datos_grupos = [data[data[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Calcular estadísticas por grupo
    stats = pd.DataFrame({
        'n': [len(datos) for datos in datos_grupos],
        'varianza': [datos.var() for datos in datos_grupos],
        'desv_std': [datos.std() for datos in datos_grupos]
    }, index=grupos)
    
    # Realizar prueba de Levene
    stat, p_value = levene(*datos_grupos)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Prueba de Levene para {len(grupos)} grupos\n"
                     f"Estadístico de Levene: {stat:.4f}\n"
                     f"Valor p: {p_value:.4f}\n\n"
                     "Estadísticas por grupo:\n")
    
    # Añadir estadísticas de cada grupo
    for grupo in grupos:
        n = stats.loc[grupo, 'n']
        var = stats.loc[grupo, 'varianza']
        std = stats.loc[grupo, 'desv_std']
        interpretacion += f"{grupo}: n={n}, varianza={var:.2f}, desv. estándar={std:.2f}\n"
    
    # Añadir conclusión
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula de igualdad de varianzas"
    interpretacion += (" (hay diferencias significativas en las varianzas entre los grupos)" 
                      if es_significativo 
                      else "\n (no hay evidencia de diferencias significativas en las varianzas)")
    
    print(interpretacion)
    return p_value

In [ ]:
resultado = compare_variances(df, 'version', 'sum_gamerounds')

Ejemplo de reporte de resultados
>Se realizó una prueba de Levene con el objetivo de evaluar si existe una diferencia significativa en la variabilidad de los tiempos de atención al cliente entre dos sucursales de una empresa de servicios (Sucursal A y Sucursal B).
La muestra incluyó 40 registros por sucursal. Los tiempos de atención se midieron en minutos. La desviación estándar en la Sucursal A fue de 4.2 minutos, mientras que en la Sucursal B fue de 6.8 minutos, lo que sugiere una mayor dispersión en esta última.
La prueba de Levene resultó en W = 5.27 con un valor p = 0.024, lo que indica que existe una diferencia estadísticamente significativa en las varianzas de los tiempos de atención entre ambas sucursales.


## Prueba t de Welch 
Si hay evidencia estadística de que las varianzas son diferentes, se recomienda hacer una prueba t de Welch (para varianzas diferentes)

In [ ]:
resultados = compare_means(df, 'version', 'sum_gamerounds', equal_var=False )

## Prueba Mann-Whitney
*¿Para qué se utiliza?*
La prueba de Mann-Whitney U es una alternativa no paramétrica a la prueba t para dos muestras independientes. Se utiliza cuando no se puede asumir la normalidad de los datos, pero se desea comparar si dos grupos independientes provienen de la misma población o si tienden a presentar valores mayores en uno que en otro.
Variables consideradas:
•	Una variable cuantitativa (de intervalo o razón), que no necesariamente sigue una distribución normal.
•	Una variable categórica binaria, que identifica los dos grupos independientes a comparar.

*¿Cómo funciona?*
La prueba ordena todos los valores de ambas muestras de menor a mayor, y luego asigna rangos. Se calcula la suma de los rangos en cada grupo y se evalúa si una de las dos muestras tiende a tener valores sistemáticamente más altos o bajos que la otra.

*Hipótesis planteadas (para prueba bilateral):*
- Hipótesis nula (H₀): Las distribuciones de los dos grupos son iguales (no hay diferencia sistemática en las posiciones o medianas).
- Hipótesis alternativa (H₁): Las distribuciones difieren (uno de los grupos tiende a tener valores mayores o menores).
También se puede formular como prueba unilateral si se desea comprobar si un grupo tiende a tener valores mayores (o menores) que el otro.

*Supuestos o requisitos principales:*
- La variable dependiente debe ser ordinal o continua.
- Las observaciones deben ser independientes dentro y entre los grupos.
- No se requiere normalidad ni homogeneidad de varianzas.
- Se asume que las distribuciones de los dos grupos tienen forma similar si se desea interpretar la prueba como una comparación de medianas.


*Criterio de decisión:*
Se calcula el estadístico U, que se convierte en un valor z (en muestras grandes) para obtener el valor p.
- Si p ≤ 0.05, se rechaza la hipótesis nula, indicando que hay diferencias significativas entre los grupos.
- Si p > 0.05, no se rechaza H₀: no hay evidencia suficiente para afirmar que los grupos difieren.


In [ ]:
from scipy.stats import mannwhitneyu

def compare_distributions(data, group_column, measure_column,
                          group1=None, group2=None, alpha = 0.05):
    """
    Realiza e interpreta una prueba de Mann-Whitney para comparar distribuciones.
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        group1: Primer grupo a comparar (si es None, usa el primero encontrado)
        group2: Segundo grupo a comparar (si es None, usa el segundo encontrado)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        valor p de la prueba
    """
    # Identificar grupos si no se especifican
    grupos = data[group_column].unique()
    if len(grupos) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    group1 = group1 if group1 is not None else grupos[0]
    group2 = group2 if group2 is not None else grupos[1]
    
    # Obtener datos de cada grupo
    datos_group1 = data[data[group_column] == group1][measure_column]
    datos_group2 = data[data[group_column] == group2][measure_column]
    
    # Calcular estadísticos descriptivos
    stats = pd.DataFrame({
        'n': [len(datos_group1), len(datos_group2)],
        'mediana': [datos_group1.median(), datos_group2.median()],
        'rango_iq': [datos_group1.quantile(0.75) - datos_group1.quantile(0.25),
                    datos_group2.quantile(0.75) - datos_group2.quantile(0.25)]
    }, index=[group1, group2])
    
    # Realizar prueba de Mann-Whitney
    statistic, p_value = mannwhitneyu(datos_group1, datos_group2)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Estadístico U: {statistic:.4f}\n"
                      f"Valor p: {p_value:.4f}\n\n"
                      "Estadísticos descriptivos:\n")
    
    for grupo in [group1, group2]:
        n = stats.loc[grupo, 'n']
        mediana = stats.loc[grupo, 'mediana']
        rango = stats.loc[grupo, 'rango_iq']
        interpretacion += f"{grupo}: mediana={mediana:.2f}, rango_IQ={rango:.2f}, n={n}\n"
    
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula" + (" (hay diferencias significativas en la distribución)" 
                                           if es_significativo else "")
    
    print(interpretacion)
    return p_value

In [ ]:
resultados = compare_distributions(df, 'version', 'sum_gamerounds')

Ejemplo de reporte de resultados:
>Se aplicó la prueba de Mann-Whitney U para comparar la satisfacción del cliente entre dos grupos de consumidores expuestos a diferentes estrategias de promoción: descuento directo y puntos de lealtad. La satisfacción se midió en una escala ordinal de 1 a 10.
La muestra incluyó 45 participantes por grupo. Dado que los datos no seguían una distribución normal (p < 0.05 en la prueba de Shapiro-Wilk), se optó por un enfoque no paramétrico.
El resultado de la prueba de Mann-Whitney fue U = 728.5, con un valor p = 0.037, lo que indica una diferencia estadísticamente significativa en los niveles de satisfacción entre ambos grupos.


## Ejercicio
Utiliza la base de datos "enigh_2020" para evaluar si ___________ por hogar en promedio es diferente entre los hogares donde el jefe de familia es hombre en comparación con los hogares donde el jefe de familia es mujer.

- Cargar los datos y librerías, explorar los datos
- Evaluar supuestos: igualdad de varianzas y normalidad (no se requiere en este caso)
- Realizar la prueba correspondiente

## Referencias
https://roirevolution.com/blog/why-ab-testing-could-save-your-marketing-strategy/